In [1]:
import pandas as pd
import numpy as np
import numpy.linalg as npl
from IPython.display import clear_output
df_2017 = pd.read_csv("data/sorted_data_from_volume_of_traffic_2017.csv")
df_sp_synx = pd.read_csv("data/london_Segment_Map5000_10000.csv")


In [2]:
# Loading data in numpy arrays
df_2017_matrix = df_2017.as_matrix()
df_sp_synx_matrix = df_sp_synx.as_matrix()

G:\apps\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
G:\apps\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
# not to calculate length every time
len_synx = len(df_sp_synx_matrix)
len_pc = len(df_2017_matrix)

In [4]:
# initializing empty array for results
combined = np.zeros((len_pc, 12))
print(combined)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
# function to calculate the distance from a point to a lane 
def diff(x1,y1,x2,y2,x3,y3):
    p1 = np.array([x1,y1])
    p2 = np.array([x2,y2])
    p3 = np.array([x3,y3])
    return npl.norm(np.cross(p2-p1, p1-p3))/npl.norm(p2-p1)
def diff2(x1,y1,x2,y2,x3,y3):
    p1[0],p1[1] = x1,y1
    p2[0],p2[1] = x2,y2
    p3[0],p3[1] = x3,y3
    return npl.norm(np.cross(p2-p1, p1-p3))/npl.norm(p2-p1)

# returns index of closest segment to a point
def find_closest_index(x, y):
    # Setting first element to min to compare it later
    x1 = df_sp_synx_matrix[0][1]
    y1 = df_sp_synx_matrix[0][2]
    x2 = df_sp_synx_matrix[0][3]
    y2 = df_sp_synx_matrix[0][4]  
    min_dist = diff(x1,y1,x2,y2,x,y)
    min_index = 0
    
    # cycling through the array to find minimal distance 
    for row in df_sp_synx_matrix:      
        dist = diff(row[1],row[2],row[3],row[4],x,y)
        if dist < min_dist:
            min_dist = dist
            min_index = row[0]
    return int(min_index)

# Test for optimizing
import time
start = time.time()
print(find_closest_index(510818, 198940))
end = time.time()
print(end-start)

In [6]:
# Create a common array contayning combined sorted data
for j in range(len_pc):
    index = find_closest_index(df_2017_matrix[j][2],df_2017_matrix[j][3])
    combined[j][0]  = df_2017_matrix[j][1]        # CP
    combined[j][1]  = df_2017_matrix[j][0]        # Year 
    combined[j][2]  = df_sp_synx_matrix[index][1] # x1 line
    combined[j][3]  = df_sp_synx_matrix[index][2] # y1 line
    combined[j][4]  = df_sp_synx_matrix[index][3] # x2 line
    combined[j][5]  = df_sp_synx_matrix[index][4] # y2 line
    combined[j][6]  = df_2017_matrix[j][2]        # x point 
    combined[j][7]  = df_2017_matrix[j][3]        # y point
    combined[j][8]  = df_2017_matrix[j][4]        # Pedal cycles
    combined[j][9]  = df_2017_matrix[j][5]        # AllMotorVehicles
    combined[j][10] = df_sp_synx_matrix[index][10]# Choice 10k
    combined[j][11] = df_sp_synx_matrix[index][11]# Choice 5k
    clear_output()
    percent = int((j/len_pc)*100)
    str = "{}%({}/{})".format(percent,j,len_pc)
    print(str)


99%(1826/1827)


In [7]:
np.savetxt("data/2_features.csv", combined, delimiter=",")